#Diabetes
dataset de clsificacion para predecir la diabestes de un paciente

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot
from scipy import optimize
import torch

%matplotlib inline

from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv("/content/drive/MyDrive/Inteligencia Artificial 1/Laboratorios/ExamenLab1/weatherAUS.csv")
df.head(10)

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No
5,2008-12-06,Albury,14.6,29.7,0.2,NaN,NaN,WNW,56.0,W,...,55.0,23.0,1009.2,1005.4,NaN,NaN,20.6,28.9,No,No
6,2008-12-07,Albury,14.3,25.0,0.0,NaN,NaN,W,50.0,SW,...,49.0,19.0,1009.6,1008.2,1.0,NaN,18.1,24.6,No,No
7,2008-12-08,Albury,7.7,26.7,0.0,NaN,NaN,W,35.0,SSE,...,48.0,19.0,1013.4,1010.1,NaN,NaN,16.3,25.5,No,No
8,2008-12-09,Albury,9.7,31.9,0.0,NaN,NaN,NNW,80.0,SE,...,42.0,9.0,1008.9,1003.6,NaN,NaN,18.3,30.2,No,Yes
9,2008-12-10,Albury,13.1,30.1,1.4,NaN,NaN,W,28.0,S,...,58.0,27.0,1007.0,1005.7,NaN,NaN,20.1,28.2,Yes,No


In [3]:
# Reemplazar valores 'NA' con NaN
df.replace('NA', np.nan, inplace=True)
# Convertir 'Date' a tipo datetime
df['Date'] = pd.to_datetime(df['Date'])
# Convertir valores numéricos a tipo float
numeric_cols = ['MinTemp', 'MaxTemp', 'Rainfall', 'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm',
                'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm',
                'Temp9am', 'Temp3pm']
df[numeric_cols] = df[numeric_cols].astype(float)
# Eliminar columnas no deseadas
df.drop(['Evaporation', 'Sunshine'], axis=1, inplace=True)
# Renombrar columnas
df.rename(columns={'WindDir9am': 'WindDirection9am', 'WindDir3pm': 'WindDirection3pm'}, inplace=True)
# Convertir 'RainToday' a variables dummy
df = pd.get_dummies(df, columns=['RainToday'], drop_first=True)

In [4]:
# Convertir las columnas categóricas en variables dummy (one-hot encoding)
categorical_cols = ['Location', 'WindGustDir', 'WindDirection9am', 'WindDirection3pm']
df = pd.get_dummies(df, columns=categorical_cols)

In [5]:
# Identificar columnas numéricas
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
# Reemplazar valores faltantes en columnas numéricas con la media
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())
# Reemplazar valores faltantes en columnas categóricas con una categoría "desconocida"
df.fillna('Desconocido', inplace=True)

In [6]:
# Convertir todas las columnas a tipo numérico
df = df.apply(pd.to_numeric, errors='ignore')
# Verificar los tipos de datos después de la conversión
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Columns: 114 entries, Date to WindDirection3pm_WSW
dtypes: bool(98), float64(14), int64(1), object(1)
memory usage: 31.4+ MB


In [7]:
print(df.shape)

(145460, 114)


In [8]:
# Calcular la cantidad de valores diferentes en una co
valores = df['RainTomorrow'].unique()
print(valores)

['No' 'Yes' 'Desconocido']


In [9]:
# Reemplazar valores en la columna 'B'
df['RainTomorrow'].replace({'Desconocido':'Yes'}, inplace=True)

In [10]:
#variables categoricas
columnas_categoricas = df.select_dtypes(include=['bool','object']).columns

In [11]:
#Procesamiento de frases o palabras
for columna in columnas_categoricas:
  le = LabelEncoder()
  df[columna] = le.fit_transform(df[columna])

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Columns: 114 entries, Date to WindDirection3pm_WSW
dtypes: float64(14), int64(100)
memory usage: 126.5 MB


Separamos los datos- Caracteristicas "**X**" y  Variable objetivo "**y**" para usarlo mas adelante

In [13]:
# Extrae la columna 'RainTomorrow' y la asigna a la variable y
y = df['RainTomorrow'].values
# Elimina la columna 'RainTomorrow' de la matriz X
X = df.drop(columns=['RainTomorrow']).values

nota: siempre normalizar

In [14]:
from sklearn.preprocessing import StandardScaler
# Crea un objeto StandardScaler
scaler = StandardScaler()
# Ajusta y transforma toda la matriz de características
X_norm = scaler.fit_transform(X)

In [15]:
# Revisamos
X_norm

array([[-1.72187528e+00,  1.89446615e-01, -4.53363105e-02, ...,
        -2.73304289e-01,  3.92322649e+00, -2.64603767e-01],
       [-1.72078953e+00, -7.53100728e-01,  2.65043084e-01, ...,
        -2.73304289e-01, -2.54892243e-01,  3.77923569e+00],
       [-1.71970377e+00,  1.10901003e-01,  3.49692009e-01, ...,
        -2.73304289e-01, -2.54892243e-01,  3.77923569e+00],
       ...,
       [ 1.67218871e+00, -1.06728318e+00,  5.18989861e-01, ...,
        -2.73304289e-01,  3.92322649e+00, -2.64603767e-01],
       [ 1.67327446e+00, -6.90264238e-01,  5.33098015e-01, ...,
        -2.73304289e-01, -2.54892243e-01, -2.64603767e-01],
       [ 1.67436022e+00,  4.25083451e-01, -5.01222327e-16, ...,
        -2.73304289e-01, -2.54892243e-01, -2.64603767e-01]])

In [16]:
from sklearn.model_selection import train_test_split
# Dividir los datos en conjuntos de entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.2, random_state=42)

In [17]:
import torch

# Verificar si hay una GPU disponible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Dispositivo:', device)

# Convertir los datos a tensores y copiar en el dispositivo
X_t = torch.from_numpy(X_train).float().to(device)
Y_t = torch.from_numpy(y_train).long().to(device)


Dispositivo: cpu


In [18]:
# Definir la arquitectura del modelo para regresión binaria
D_in, H, D_out = X.shape[1], 50, 1

In [19]:
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
    torch.nn.Sigmoid()  # Función de activación sigmoide para regresión binaria
)

In [20]:
model.cpu()

Sequential(
  (0): Linear(in_features=113, out_features=50, bias=True)
  (1): ReLU()
  (2): Linear(in_features=50, out_features=1, bias=True)
  (3): Sigmoid()
)

In [21]:
# Función de pérdida para regresión binaria
criterion = torch.nn.BCELoss()

In [22]:
# Hiperparámetros
epochs = 100
lr = 0.1
long_each = 10

In [23]:
# Optimizador
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [24]:
# Bucle de entrenamiento
for e in range(1, epochs + 1):
    # Forward
    y_pred = model(X_t)

    # Calcular la pérdida
    loss = criterion(y_pred.view(-1), Y_t.float())

    # Reiniciar los gradientes
    optimizer.zero_grad()

    # Backpropagation
    loss.backward()

    # Actualizar pesos
    optimizer.step()

    if e % long_each == 0:
        print(f"Epoch {e}/{epochs} Loss: {loss.item()}")


Epoch 10/100 Loss: 0.5571497678756714
Epoch 20/100 Loss: 0.5237777233123779
Epoch 30/100 Loss: 0.5035014748573303
Epoch 40/100 Loss: 0.48746949434280396
Epoch 50/100 Loss: 0.47383663058280945
Epoch 60/100 Loss: 0.46202757954597473
Epoch 70/100 Loss: 0.451803594827652
Epoch 80/100 Loss: 0.4430088996887207
Epoch 90/100 Loss: 0.4355120062828064
Epoch 100/100 Loss: 0.4291691184043884


In [25]:
# Función para evaluar el modelo
def evaluate(x):
    model.eval()
    y_pred = model(x)
    return (y_pred > 0.5).float().cpu().numpy()

In [29]:
from sklearn.metrics import accuracy_score
# Evaluar el modelo en el conjunto de prueba
X_train_t = torch.from_numpy(X_train).float().cpu()
y_pred = evaluate(X_train_t)
accuracy = accuracy_score(y_train, y_pred)*100
print(f"Accuracy: {accuracy:.2f}%")

Accuracy: 80.05%


In [27]:
from sklearn.metrics import accuracy_score
# Evaluar el modelo en el conjunto de prueba
X_test_t = torch.from_numpy(X_test).float().cpu()
y_pred = evaluate(X_test_t)
accuracy = accuracy_score(y_test, y_pred)*100
print(f"Accuracy: {accuracy:.2f}%")

Accuracy: 79.75%


In [28]:
# Evalúa el modelo en el conjunto de prueba
model.eval()
y_pred = model(X_test_t)
y_pred_labels = torch.argmax(y_pred, axis=1)
print("Etiqueta predicha para la primera muestra:", y_pred_labels[0])

Etiqueta predicha para la primera muestra: tensor(0)
